# Imports and configs

In [23]:
from sklearn.model_selection import KFold, cross_val_score, cross_validate
from sklearn.metrics import make_scorer
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import pearsonr

from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.base import clone

from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import Ridge
from lightgbm import LGBMRegressor
from xgboost import XGBRegressor

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
import optuna
import joblib
import glob
import gc
import time

warnings.filterwarnings("ignore")

In [2]:
class CFG:
    train_path = "input/train.parquet"
    test_path = "input/test.parquet"
    sample_sub_path = "input/sample_submission.csv"

    target = "label"
    features_main = ['bid_qty', 'ask_qty', 'buy_qty', 'sell_qty', 'volume']
    features_x = ['X' + str(i) for i in range(1,890+1)]

    n_folds = 5
    seed = 42

    run_optuna = True
    n_optuna_trials = 100 

# Data loading

Discussionで拾ってきたやつ。データ量を圧縮して分析効率を上げることができる。
https://www.kaggle.com/competitions/drw-crypto-market-prediction/discussion/580485

In [3]:
def reduce_mem_usage(dataframe, dataset):    
    print('Reducing memory usage for:', dataset)
    initial_mem_usage = dataframe.memory_usage().sum() / 1024**2

    for col in dataframe.columns:
        col_type = dataframe[col].dtype

        c_min = dataframe[col].min()
        c_max = dataframe[col].max()
        if str(col_type)[:3] == 'int':
            if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                dataframe[col] = dataframe[col].astype(np.int8)
            elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                dataframe[col] = dataframe[col].astype(np.int16)
            elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                dataframe[col] = dataframe[col].astype(np.int32)
            elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                dataframe[col] = dataframe[col].astype(np.int64)
        else:
            if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                dataframe[col] = dataframe[col].astype(np.float16)
            elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                dataframe[col] = dataframe[col].astype(np.float32)
            else:
                dataframe[col] = dataframe[col].astype(np.float64)

    final_mem_usage = dataframe.memory_usage().sum() / 1024**2
    print('--- Memory usage before: {:.2f} MB'.format(initial_mem_usage))
    print('--- Memory usage after: {:.2f} MB'.format(final_mem_usage))
    print('--- Decreased memory usage by {:.1f}%\n'.format(100 * (initial_mem_usage - final_mem_usage) / initial_mem_usage))

    return dataframe


In [4]:
train = pd.read_parquet(CFG.train_path).reset_index(drop=True)
test = pd.read_parquet(CFG.test_path).reset_index(drop=True)

train = reduce_mem_usage(train, "train")
test = reduce_mem_usage(test, "test")

X = train.drop(CFG.target, axis=1)
y = train[CFG.target]
X_test = test.drop(CFG.target, axis=1)

Reducing memory usage for: train
--- Memory usage before: 3594.93 MB
--- Memory usage after: 961.92 MB
--- Decreased memory usage by 73.2%

Reducing memory usage for: test
--- Memory usage before: 3678.76 MB
--- Memory usage after: 984.36 MB
--- Decreased memory usage by 73.2%



# Preprocessing

In [5]:
train.head()

bid_qty    ask_qty   buy_qty   sell_qty   volume        X1        X2  \
0  15.281250   8.421875  176.3750   44.96875  221.375  0.121277 -0.417725   
1  38.593750   2.335938  526.0000  322.00000  848.000  0.302734 -0.049561   
2   0.441895  60.250000  159.2500  136.37500  295.500  0.167480 -0.291260   
3   4.863281  21.015625  335.7500  124.93750  460.750  0.072937 -0.436523   
4  27.156250   3.451172   98.4375   44.40625  142.875  0.173828 -0.213501   

         X3        X4        X5  ...      X882      X883      X884      X885  \
0  0.005398  0.125977  0.058350  ...  1.925781  1.847656  0.005676  0.190796   
1  0.356689  0.481201  0.237915  ...  1.928711  1.849609  0.005226  0.184692   
2  0.083130  0.206909  0.101746  ...  1.927734  1.849609  0.004795  0.178711   
3 -0.102478  0.017548  0.007149  ...  1.928711  1.849609  0.004398  0.172974   
4  0.096069  0.215698  0.107117  ...  1.926758  1.848633  0.004009  0.167358   

       X886      X887      X888      X889      X890     label  
0  0.369629  0.377686  0.210205  0.159180  0.530762  0.562500  
1  0.363525  0.374512  0.209595  0.158936  0.530273  0.533691  
2  0.357666  0.371338  0.208984  0.158691  0.529785  0.546387  
3  0.351807  0.368408  0.208374  0.158569  0.529297  0.357666  
4  0.345947  0.365234  0.207886  0.158325  0.529297  0.362549  

[5 rows x 896 columns]

In [7]:
features = [
    "X863", "X856", "X344", "X598", "X862", "X385", "X852", "X603", "X860", "X674",
    "X415", "X345", "X137", "X855", "X174", "X302", "X178", "X532", "X168", "X612",
    "bid_qty", "ask_qty", "buy_qty", "sell_qty", "volume"
]

X = X[features]
X_test = X_test[features]

# Training base model

In [8]:
# 共通設定
kf = KFold(n_splits=CFG.n_folds, shuffle=True, random_state=CFG.seed)

In [9]:
def pearson_score(y_true, y_pred):
    """Pearson相関スコア"""
    return pearsonr(y_true, y_pred)[0]

In [10]:

def run_cross_validation(model, X, y, model_name="Model", cv_folds=None):
    """
    fold別詳細結果表示対応クロスバリデーション関数
    
    Parameters:
    -----------
    model : sklearn compatible model
        学習させるモデル
    X : pandas.DataFrame
        特徴量データ
    y : pandas.Series
        目的変数
    model_name : str
        モデル名
    cv_folds : KFold object
        クロスバリデーション設定
    
    Returns:
    --------
    dict : CV結果の辞書
    """
    
    if cv_folds is None:
        cv_folds = KFold(n_splits=CFG.n_folds, shuffle=False, random_state=None)
    
    print(f"=== {model_name} Cross Validation 開始 ===")
    
    # 結果保存用
    fold_results = []
    rmse_scores = []
    r2_scores = []
    pearson_scores = []
    times = []
    
    # 各fold実行
    for fold, (train_idx, val_idx) in enumerate(cv_folds.split(X, y)):
        start_time = time.time()
        
        # データ分割
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
        
        # モデル学習
        model.fit(X_train, y_train)
        
        # 予測
        y_pred = model.predict(X_val)
        
        # 評価指標計算
        rmse = np.sqrt(mean_squared_error(y_val, y_pred))
        r2 = r2_score(y_val, y_pred)
        pearson = pearsonr(y_val, y_pred)[0]
        
        # 実行時間
        fold_time = time.time() - start_time
        
        # 結果保存
        rmse_scores.append(rmse)
        r2_scores.append(r2)
        pearson_scores.append(pearson)
        times.append(fold_time)
        
        fold_results.append({
            'fold': fold,
            'rmse': rmse,
            'r2': r2,
            'pearson': pearson,
            'time': fold_time
        })
        
        # fold別結果表示
        print(f"--- Fold {fold} - pearsonr: {pearson:.6f} - Time: {fold_time:.2f} s")
    
    # 全体統計
    results = {
        'fold_results': fold_results,
        'rmse': {
            'mean': np.mean(rmse_scores),
            'std': np.std(rmse_scores),
            'scores': rmse_scores
        },
        'r2': {
            'mean': np.mean(r2_scores),
            'std': np.std(r2_scores),
            'scores': r2_scores
        },
        'pearson': {
            'mean': np.mean(pearson_scores),
            'std': np.std(pearson_scores),
            'scores': pearson_scores
        },
        'time': {
            'mean': np.mean(times),
            'total': np.sum(times)
        }
    }
    
    # 全体結果表示
    print(f"\n{model_name} Overall Results:")
    print(f"  RMSE: {results['rmse']['mean']:.6f} ± {results['rmse']['std']:.6f}")
    print(f"  R²: {results['r2']['mean']:.6f} ± {results['r2']['std']:.6f}")
    print(f"  Pearson: {results['pearson']['mean']:.6f} ± {results['pearson']['std']:.6f}")
    print(f"  Total Time: {results['time']['total']:.2f} s")
    print()
    
    return results

## LightGBM (gbdt)

In [11]:
print("📊 LightGBM学習開始")
lgbm_model = LGBMRegressor(
    objective='regression',
    device='gpu',
    gpu_platform_id=0,
    gpu_device_id=0,
    learning_rate=0.1,
    num_leaves=64,
    max_depth=7,
    n_estimators=1000,
    verbose=-1
)

lgbm_results = run_cross_validation(
    model=lgbm_model,
    X=X,
    y=y,
    model_name="LightGBM",
    cv_folds=kf
)

📊 LightGBM学習開始
=== LightGBM Cross Validation 開始 ===
--- Fold 0 - pearsonr: 0.953549 - Time: 12.91 s
--- Fold 1 - pearsonr: 0.955484 - Time: 10.68 s
--- Fold 2 - pearsonr: 0.954562 - Time: 11.74 s
--- Fold 3 - pearsonr: 0.953501 - Time: 11.62 s
--- Fold 4 - pearsonr: 0.952899 - Time: 11.41 s

LightGBM Overall Results:
  RMSE: 0.314232 ± 0.002747
  R²: 0.903168 ± 0.001699
  Pearson: 0.953999 ± 0.000915
  Total Time: 58.36 s



## XGBoost

In [12]:
print("📊 XGBoost学習開始")
xgb_model = XGBRegressor(
    objective='reg:squarederror',
    tree_method='gpu_hist',
    learning_rate=0.1,
    max_depth=7,
    n_estimators=1000,
    verbosity=0
)

xgb_results = run_cross_validation(
    model=xgb_model,
    X=X,
    y=y,
    model_name="XGBoost",
    cv_folds=kf
)

📊 XGBoost学習開始
=== XGBoost Cross Validation 開始 ===
--- Fold 0 - pearsonr: 0.966268 - Time: 7.56 s
--- Fold 1 - pearsonr: 0.966772 - Time: 7.17 s
--- Fold 2 - pearsonr: 0.966472 - Time: 6.97 s
--- Fold 3 - pearsonr: 0.966839 - Time: 6.88 s
--- Fold 4 - pearsonr: 0.965293 - Time: 6.83 s

XGBoost Overall Results:
  RMSE: 0.267518 ± 0.001618
  R²: 0.929816 ± 0.001222
  Pearson: 0.966329 ± 0.000557
  Total Time: 35.40 s



## NN

In [13]:
# CPU用に最適化されたMLP
print("🏃‍♂️ 最適化MLP (CPU) 学習開始")

optimized_mlp = Pipeline([
    ('scaler', StandardScaler()),
    ('mlp', MLPRegressor(
        hidden_layer_sizes=(128, 64),  # 層を減らして高速化
        activation='relu',
        solver='adam',
        alpha=0.01,  # 正則化を強めて過学習抑制
        batch_size=200,  # バッチサイズを大きくして高速化
        learning_rate='adaptive',
        learning_rate_init=0.01,
        max_iter=300,  # 反復数を減らして高速化
        random_state=42,
        early_stopping=True,
        validation_fraction=0.1,
        n_iter_no_change=20  # 早期停止を早めて高速化
    ))
])

optimized_mlp_results = run_cross_validation(
    model=optimized_mlp,
    X=X,
    y=y,
    model_name="Optimized MLP (CPU)",
    cv_folds=kf
)

🏃‍♂️ 最適化MLP (CPU) 学習開始
=== Optimized MLP (CPU) Cross Validation 開始 ===
--- Fold 0 - pearsonr: 0.791054 - Time: 170.53 s
--- Fold 1 - pearsonr: 0.800777 - Time: 239.37 s
--- Fold 2 - pearsonr: 0.781951 - Time: 189.64 s
--- Fold 3 - pearsonr: 0.788265 - Time: 237.22 s
--- Fold 4 - pearsonr: 0.761368 - Time: 180.11 s

Optimized MLP (CPU) Overall Results:
  RMSE: 0.627147 ± 0.012882
  R²: 0.613963 ± 0.020330
  Pearson: 0.784683 ± 0.013141
  Total Time: 1016.88 s



# Ensemble

In [14]:

def create_ensemble_predictions(trained_models, model_names, X, y, X_test, cv_folds):
    """
    学習済みモデルを使ってアンサンブル用の予測を作成
    
    Parameters:
    -----------
    trained_models : dict
        学習済みモデルの辞書 {'model_name': model}
    model_names : list
        使用するモデル名のリスト
    X, y : DataFrame, Series
        学習データ
    X_test : DataFrame
        テストデータ
    cv_folds : KFold
        クロスバリデーション設定
        
    Returns:
    --------
    oof_predictions : DataFrame
        Out-of-Fold予測データフレーム
    test_predictions : DataFrame
        テストデータ予測データフレーム
    """
    
    print("🔄 学習済みモデルでアンサンブル予測作成中...")
    
    # 結果保存用辞書
    oof_preds = {}
    test_preds = {}
    
    for model_name in model_names:
        print(f"  📊 {model_name} のアンサンブル予測作成中...")
        
        # OOF予測用配列初期化
        oof_pred = np.zeros(len(X))
        test_pred_folds = []
        
        # 各foldで予測
        for fold, (train_idx, val_idx) in enumerate(cv_folds.split(X, y)):
            # データ分割
            X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
            y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]
            
            # 学習済みモデルをクローンして再学習
            # (学習済みパラメータを使用するため、同じ設定で学習)
            fold_model = clone(trained_models[model_name])
            fold_model.fit(X_train, y_train)
            
            # OOF予測
            oof_pred[val_idx] = fold_model.predict(X_val)
            
            # テストデータ予測
            test_pred_folds.append(fold_model.predict(X_test))
            
        # テストデータ予測の平均
        test_pred = np.mean(test_pred_folds, axis=0)
        
        # 結果保存
        oof_preds[model_name] = oof_pred
        test_preds[model_name] = test_pred
        
        # OOF性能確認
        oof_pearson = pearsonr(y, oof_pred)[0]
        print(f"    ✅ {model_name} OOF Pearson: {oof_pearson:.6f}")
    
    # DataFrameに変換
    oof_df = pd.DataFrame(oof_preds)
    test_df = pd.DataFrame(test_preds)
    
    return oof_df, test_df

In [24]:

def optimize_ridge_alpha(oof_predictions, y, alphas=None):
    """
    Ridge回帰のalphaパラメータを最適化
    
    Parameters:
    -----------
    oof_predictions : DataFrame
        OOF予測データフレーム
    y : Series
        目的変数
    alphas : list
        試行するalphaの値リスト
        
    Returns:
    --------
    best_alpha : float
        最適なalpha値
    """
    
    if alphas is None:
        alphas = [0.01, 0.1, 0.5, 1.0, 2.0, 5.0, 10.0, 20.0]
    
    print("🔍 Ridge alpha パラメータ最適化中...")
    
    best_alpha = None
    best_score = -np.inf
    
    for alpha in alphas:
        ridge = Ridge(alpha=alpha, random_state=CFG.seed)
        
        # クロスバリデーションでスコア計算
        scores = cross_val_score(
            ridge, oof_predictions, y, 
            cv=5, 
            scoring='neg_mean_squared_error'
        )
        mean_score = scores.mean()
        
        print(f"  alpha={alpha:>6}: CV Score = {-mean_score:.6f}")
        
        if mean_score > best_score:
            best_score = mean_score
            best_alpha = alpha
    
    print(f"✅ 最適alpha: {best_alpha}")
    return best_alpha

In [16]:
def ridge_ensemble_final(oof_predictions, test_predictions, y, alpha=None):
    """
    Ridgeでアンサンブルモデルを学習・予測
    
    Parameters:
    -----------
    oof_predictions : DataFrame
        OOF予測データフレーム
    test_predictions : DataFrame  
        テストデータ予測データフレーム
    y : Series
        目的変数
    alpha : float
        Ridge正則化パラメータ（Noneの場合は最適化）
        
    Returns:
    --------
    ridge_model : Ridge
        学習済みRidgeモデル
    final_predictions : ndarray
        最終アンサンブル予測
    model_weights : ndarray
        各モデルの重み
    """
    
    print("🏗️ Ridge Ensemble 最終学習開始")
    
    # alpha最適化（指定されていない場合）
    if alpha is None:
        alpha = optimize_ridge_alpha(oof_predictions, y)
    
    # Ridge学習
    ridge_model = Ridge(alpha=alpha, random_state=CFG.seed)
    ridge_model.fit(oof_predictions, y)
    
    # 最終予測
    final_predictions = ridge_model.predict(test_predictions)
    
    # モデル重み取得
    model_weights = ridge_model.coef_
    
    print(f"\n📈 Ridge Ensemble Results (alpha={alpha}):")
    print(f"  切片: {ridge_model.intercept_:.6f}")
    print(f"  モデル重み:")
    for name, weight in zip(oof_predictions.columns, model_weights):
        print(f"    {name}: {weight:.6f}")
    
    # OOFでの性能確認
    oof_ensemble_pred = ridge_model.predict(oof_predictions)
    oof_pearson = pearsonr(y, oof_ensemble_pred)[0]
    oof_rmse = np.sqrt(mean_squared_error(y, oof_ensemble_pred))
    
    print(f"\n✅ Ridge Ensemble OOF Performance:")
    print(f"  Pearson: {oof_pearson:.6f}")
    print(f"  RMSE: {oof_rmse:.6f}")
    
    return ridge_model, final_predictions, model_weights

In [17]:
def plot_weights(weights, model_names, title="Model Weights", figsize=(10, 6)):
    """
    アンサンブルモデルの重みを可視化する関数
    
    Parameters:
    -----------
    weights : array-like
        モデルの重み（係数）
    model_names : list
        モデル名のリスト
    title : str
        グラフのタイトル
    figsize : tuple
        図のサイズ
    """
    
    # 入力データの検証
    if len(weights) != len(model_names):
        raise ValueError(f"重みの数({len(weights)})とモデル名の数({len(model_names)})が一致しません")
    
    # weightsが2次元配列の場合は1次元に変換
    if isinstance(weights, np.ndarray) and weights.ndim == 2:
        weights = weights.flatten()
    
    # 降順でソート
    sorted_indices = np.argsort(weights)[::-1]
    sorted_coeffs = np.array(weights)[sorted_indices]
    sorted_model_names = np.array(model_names)[sorted_indices]
    
    # グラフサイズを動的に調整
    height = max(len(model_names) * 0.5, 4)  # 最小4, モデル数に応じて調整
    
    plt.figure(figsize=(figsize[0], height))
    ax = sns.barplot(x=sorted_coeffs, y=sorted_model_names, palette="RdYlGn_r")
    
    # 数値ラベルの追加
    for i, (value, name) in enumerate(zip(sorted_coeffs, sorted_model_names)):
        if value >= 0:
            ax.text(value + 0.01, i, f"{value:.3f}", va="center", ha="left", color="black", fontweight='bold')
        else:
            ax.text(value - 0.01, i, f"{value:.3f}", va="center", ha="right", color="black", fontweight='bold')
    
    # x軸の範囲を調整（ラベルが見えるように）
    xlim = ax.get_xlim()
    x_range = xlim[1] - xlim[0]
    ax.set_xlim(xlim[0] - 0.15 * x_range, xlim[1] + 0.15 * x_range)
    
    # グラフの装飾
    plt.title(title, fontsize=14, fontweight='bold')
    plt.xlabel("Weight (係数)", fontsize=12)
    plt.ylabel("Model (モデル)", fontsize=12)
    
    # 0の線を追加（負の重みがある場合）
    if np.any(sorted_coeffs < 0):
        plt.axvline(x=0, color='black', linestyle='--', alpha=0.5, linewidth=1)
    
    plt.tight_layout()
    plt.show()
    
    # 重み情報をテキストでも表示
    print(f"\n📊 {title}")
    print("="*50)
    for name, weight in zip(sorted_model_names, sorted_coeffs):
        print(f"{name:15}: {weight:8.6f}")
    print("="*50)
    total_abs_weight = np.sum(np.abs(sorted_coeffs))
    print(f"{'Total |Weight|':15}: {total_abs_weight:8.6f}")

In [19]:
# 学習済みモデルを辞書にまとめる
trained_models = {
    'LightGBM': lgbm_model,
    'XGBoost': xgb_model, 
    'MLP': optimized_mlp
}

model_names = ['LightGBM', 'XGBoost', 'MLP']

print("🚀 Ridge Ensemble Pipeline 開始")
print(f"📊 使用モデル: {model_names}")

# 1. アンサンブル用予測作成
oof_predictions, test_predictions = create_ensemble_predictions(
    trained_models=trained_models,
    model_names=model_names,
    X=X,
    y=y,
    X_test=X_test,
    cv_folds=kf
)

print(f"\n📋 アンサンブル予測データ形状:")
print(f"  OOF予測: {oof_predictions.shape}")
print(f"  テスト予測: {test_predictions.shape}")

🚀 Ridge Ensemble Pipeline 開始
📊 使用モデル: ['LightGBM', 'XGBoost', 'MLP']
🔄 学習済みモデルでアンサンブル予測作成中...
  📊 LightGBM のアンサンブル予測作成中...
    ✅ LightGBM OOF Pearson: 0.954004
  📊 XGBoost のアンサンブル予測作成中...
    ✅ XGBoost OOF Pearson: 0.966333
  📊 MLP のアンサンブル予測作成中...
    ✅ MLP OOF Pearson: 0.783780

📋 アンサンブル予測データ形状:
  OOF予測: (525887, 3)
  テスト予測: (538150, 3)


In [25]:
# 2. Ridge Ensemble学習・予測
ridge_model, ensemble_predictions, model_weights = ridge_ensemble_final(
    oof_predictions=oof_predictions,
    test_predictions=test_predictions,
    y=y,
    alpha=None  # 自動最適化
)

🏗️ Ridge Ensemble 最終学習開始
🔍 Ridge alpha パラメータ最適化中...
  alpha=  0.01: CV Score = 0.067313
  alpha=   0.1: CV Score = 0.067313
  alpha=   0.5: CV Score = 0.067313
  alpha=   1.0: CV Score = 0.067313
  alpha=   2.0: CV Score = 0.067313
  alpha=   5.0: CV Score = 0.067313
  alpha=  10.0: CV Score = 0.067313
  alpha=  20.0: CV Score = 0.067313
✅ 最適alpha: 20.0

📈 Ridge Ensemble Results (alpha=20.0):
  切片: -0.002180
  モデル重み:
    LightGBM: 0.083016
    XGBoost: 1.014042
    MLP: -0.033090

✅ Ridge Ensemble OOF Performance:
  Pearson: 0.966485
  RMSE: 0.259269


In [27]:
plot_weights(
    weights=model_weights,  # 1次元配列
    model_names=list(oof_predictions.columns),  # DataFrameのカラム名から取得
    title="Ridge Coefficients"
)


📊 Ridge Coefficients
XGBoost        : 1.014042
LightGBM       : 0.083016
MLP            : -0.033090
Total |Weight| : 1.130148


# Submission

In [29]:
# スコア計算
ridge_oof_score = pearson_score(y, ridge_model.predict(oof_predictions))

# 提出ファイル
sub = pd.read_csv(CFG.sample_sub_path)
sub["prediction"] = ensemble_predictions
sub.to_csv(f"output/sub_ridge_{ridge_oof_score:.6f}.csv", index=False)


✅ 保存完了: sub_ridge_0.966485.csv


In [1]:
sub.head()

NameError: name 'sub' is not defined